In [47]:
!pip install langchain
!pip install tqdm
!pip install langchain-pinecone

!pip install tqdm
!pip install langchain pinecone-client google-generativeai openai tqdm

In [48]:
from google.colab import userdata
import os
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
PINECONE_ENVIRONMENT = 'us-east-1'


from pinecone import Pinecone, ServerlessSpec


pc = Pinecone(
    api_key=PINECONE_API_KEY
)

# Check if the index exists; if not, create it
index_name = "responseindex"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",  # Choose the metric: cosine, euclidean, or dotproduct
        spec=ServerlessSpec(
            cloud="aws",
            region=PINECONE_ENVIRONMENT  # Use your environment's region
        )
    )

# # Connect to the index
index = pc.Index(name=index_name)
print(f"Successfully connected to index: {index_name}")


Successfully connected to index: responseindex


In [16]:
!pip install -q langchain-google-genai
!pip install python-dotenv

In [37]:
import os
from dotenv import load_dotenv

load_dotenv()
PINECONE_API_KEY=os.getenv("PINECODE_API_KEY")


In [32]:
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings

# os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY # Assuming PINECONE_API_KEY is already defined


embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",  # Specify the desired embedding model
    api_key=GOOGLE_API_KEY
)

In [11]:
!pip install langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [12]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load documents
loader = TextLoader("/content/document.txt")  # Replace with your file
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

In [33]:
from tqdm import tqdm

# Create embeddings and upload to Pinecone
for doc in tqdm(docs):

    vector = embeddings.embed_query(doc.page_content)

    # Modify the upsert to use a dictionary for metadata
    index.upsert([{
        "id": doc.metadata["source"],  # Use "id" instead of the first tuple element
        "values": vector,  # Use "values" for the vector
        "metadata": {
            "text": doc.page_content,  # Add the text as part of metadata
            "source": doc.metadata["source"]  # Include the source
        }
    }])

100%|██████████| 29/29 [00:13<00:00,  2.19it/s]


In [51]:
from langchain.vectorstores import Pinecone

# Use from_existing_index to load from an existing index
retriever = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings, text_key="text")

In [52]:
from langchain_google_genai import ChatGoogleGenerativeAI

gemini_model = ChatGoogleGenerativeAI(api_key=GOOGLE_API_KEY,model="gemini-1.5-flash", temperature=0.7)

In [53]:
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA

# Create a vector store using the Pinecone index
vectorstore = PineconeVectorStore(
    index_name=index_name,
    embedding=embeddings
)

# Create the retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})  # Retrieve top 2 most similar documents

# Create the QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=gemini_model,
    chain_type="stuff",
    retriever=retriever,  # Pass the retriever here
    return_source_documents=True  # Optional: to get the source documents used in the response
)

In [54]:
print("Welcome To Zeeshan's GenAI World")
print('='*40)

query = "What is the Future of Agentic AI?"
print('Human Message:',query)
response = qa_chain.invoke(query)

# Print the answer
print("Agent Message:", response['result'])

# Print the source documents
# print("\n According To Given Information:")
for doc in response['source_documents']:
    print(f"- {doc.page_content[:200]}...")  # Print first 200 characters of each source document

Welcome To Zeeshan's GenAI World
Human Message: What is the Future of Agentic AI?
Agent Message: Based on the provided text, the future of Agentic AI involves the development of smarter, more adaptive systems capable of autonomously generating solutions, creating content, and interacting with the world in more natural, human-like ways.

- The progress in Generative AI and Agentic AI will lead to smarter, more adaptive systems that can autonomously generate solutions, create content, and interact with the world in more natural, human-li...


In [55]:
print("Welcome To Zeeshan's Coding World")
print('='*40)

query = "What is the Future of AI Agents?"
print('Human Message:',query)
response = qa_chain.invoke(query)

# Print the answer
print("Agent Message:", response['result'])

# Print the source documents
print("\n According To Given Information:")
for doc in response['source_documents']:
    print(f"- {doc.page_content[:200]}...")  # Print first 200 characters of each source document

Welcome To Zeeshan's Coding World
Human Message: What is the Future of AI Agents?
Agent Message: Based on the provided text, the future of AI agents involves smarter, more adaptive systems capable of autonomously generating solutions, creating content, and interacting with the world in more natural, human-like ways.


 According To Given Information:
- The progress in Generative AI and Agentic AI will lead to smarter, more adaptive systems that can autonomously generate solutions, create content, and interact with the world in more natural, human-li...
